完成苏宁电子书的数据抓取,包括大分类，小分类，图书的名字，连接，作者，出版社，简介等信息，url:http://snbook.suning.com/web/trd-fl/999999/0.htm


In [ ]:
import scrapy
import re
from copy import deepcopy
import urllib.parse


class SuningSpider(scrapy.Spider):
    name = 'suning'
    allowed_domains = ['suning.com']
    start_urls = ['http://snbook.suning.com/web/trd-fl/999999/0.htm']

    def parse(self, response):
        # 获取大分类
        li_list = response.xpath("//ul[@class='ulwrap']/li")
        for li in li_list:
            item = {}
            # 大分类的名字
            item["b_cate"] = li.xpath("./div[1]/a/text()").extract_first()
            # 小分类分组
            a_list = li.xpath("./div[2]/a")
            for a in a_list:
                # 小分类的名字
                item["s_cate"] = a.xpath("./text()").extract_first()
                # 小分类的url地址
                item["s_href"] = a.xpath("./@href").extract_first()
                # print(item)
                if item["s_href"] is not None:
                    item["s_href"] = urllib.parse.urljoin(
                        response.url, item["s_href"])
                    # 构造小分类url地址的请求
                    yield scrapy.Request(
                        item["s_href"],
                        callback=self.parse_book_list,
                        meta={"item": deepcopy(item)}
                    )

    def parse_book_list(self, response):  # 提取列表页的数据
        item = response.meta["item"]
        # 获取数据列表页数据的分组
        li_list = response.xpath(
            "//div[@class='filtrate-books list-filtrate-books']/ul/li")
        for li in li_list:
            item["book_name"] = li.xpath(
                ".//div[@class='book-title']/a/text()").extract_first()
            item["book_href"] = li.xpath(
                ".//div[@class='book-title']/a/@href").extract_first()
            item["book_author"] = li.xpath(
                ".//div[@class='book-author']/a/text()").extract_first()
            # 进入图书详情页
            yield scrapy.Request(
                item["book_href"],
                callback=self.parse_book_price,
                meta={"item": deepcopy(item)}
            )
        # 列表页的翻页
        # 获取总的页码数
        page_count = re.findall("var pagecount=(.*?);",
                                response.body.decode())[0]
        # 获取当前页码数
        current_page = re.findall(
            "var currentPage=(.*?);", response.body.decode())[0]
        if int(current_page) < int(page_count):
            # 下一页的url地址
            next_url = item["s_href"] + \
                "?pageNumber={}&sort=0".format(int(current_page)+1)
            # 构造请求
            yield scrapy.FormRequest(
                next_url,
                callback=self.parse_book_list,
                formdata={"ajaxFlag": "true"},
                meta={"item": item}
            )

    def parse_book_price(self, response):  # 提取详情页的价格
        item = response.meta["item"]
        item["book_price"] = re.findall(
            "\"bp\":'(.*?)',", response.body.decode())[0]
        print(item)
